# Sample code for performing obstacle avoidance #

Import necessary packages.

In [1]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

Set your group number and camera number.

In [2]:
group_number = 18

# Possibly try 0, 1, 2 ...
camera = 1

## Tune the red filtering ##

You can use the following cell to test and visualize the red filtering. This cell *not* make the drone fly. It will connect to the CrazyFlie camera and perform red filtering on the live video feed. You should use this cell to tune the HSV intervals, and then copy/paste your tuned intervals into the __check_contours__ function below. When tuning the intervals, keep in mind that the lighting in the environment can matter.

In [ ]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    ''' red bounds:
    lb1 = (150, 25, 130)
    ub1 = (180, 150, 240)
    lb2 = (0, 25, 130)
    ub2 = (20, 150, 240)
    '''
    lb1 = (100, 25, 130)
    ub1 = (130, 150, 240)
    lb2 = (110, 25, 130)  # Adjust these if a second range is needed
    ub2 = (140, 150, 240)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Compute
    #cv2.imshow('mask', mask)  
    cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

## Helper functions ##

The following cell contains some sample functions which will be useful.

In particular, __check_contours__ and __find_greatest_contour__ will perform red filtering on the live camera feed and identify the obstacles. The red filtering is controlled by setting HSV intervals in the __check_contours__ function. Note that the intervals will require tuning and may vary on different drones/cameras.

The __adjust_position__ function can also be modified for performing obstacle avoidance.

In [27]:
# HELPER FUNCTIONS
# FROM LAB 8

# returns center of image object
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    center_x = (detection[3] + detection[5]) / 2.0 - 0.5
    center_y = (detection[4] + detection[6]) / 2.0 - 0.5
    return (center_x, center_y)

def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

# retruns image closest to the center of the frame
def closest_detection(detections):
    """TODO: Find the detection closest to the image center"""
    # Loop through and find the detection that is closest to the image center
    # You can use the detection_center function above to find the center of the detected object
    # Note that the origin (i.e., (x,y) = (0,0)) corresponds to the center of the image. So you can
    # use the "norm" function above to find the detection that is closest to the center.
    # Return the det that corresponds to the closest detection to the image center.
    # If nothing is detected, return None.
    if not detections:
        return None
    
    minval = float('inf')
    detection = detections[0]
    for det in detections:
        center = detection_center(det)
        vec_length = norm(center)
        if vec_length < minval:
            minval = vec_length
            detection = det
    return detection


# Sort through contours in the image
def find_greatest_contour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index


# Find contours in the image; returns are_there_contours, largest_area, center_x 
def check_contours(image, lb1, ub1, lb2, ub2):

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.

    largest_area = 0
    center_x = 0
    

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        
        are_there_contours = True
        largest_area, largest_contour_index = find_greatest_contour(contours)
        largest_contour = contours[largest_contour_index]
        moment = cv2.moments(largest_contour)
        
        if moment["m00"] == 0:
            are_there_contours = False
        else:
            center_x = int(moment["m10"]/moment["m00"])

    return True, largest_area, center_x 


# FROM SAMPLE CODE
    
# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


# Ascend and hover at 1m:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at .85 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.85)
        time.sleep(0.1)
    return


# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending:')
    # Hover at 0.85 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.85)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return


In [40]:
# CONTROLLER

# Follow the setpoint sequence trajectory:
# def adjust_position(cf, current_y):

#     print('Adjusting position')

#     steps_per_meter = int(10)
#     # Set the number here (the iterations of the for-loop) to the number of side steps.
#     # You may choose to tune the number and size of the steps.
#     for i in range(3): 
#         current_y = current_y - 1.0/float(steps_per_meter)
#         position = [0, current_y, 0.5, 0.0]

#         print('Setting position {}'.format(position))
#         for i in range(10):
#             cf.commander.send_position_setpoint(position[0],
#                                                 position[1],
#                                                 position[2],
#                                                 position[3])
#             time.sleep(0.1)

#     cf.commander.send_stop_setpoint()
#     # Make sure that the last packet leaves before the link is closed.
#     # The message queue is not flushed before closing.
#     time.sleep(0.1)
#     return current_y

# Follow the setpoint sequence trajectory:
def adjust_position(cf, image, box_x, box_y, box_width, box_height, current_x, current_y):
    x = current_x
    y = current_y
    delta = 0.02
    
    _, width, _ = image.shape
    margin = .15 * width

    r_lb1 = (150, 25, 130)
    r_ub1 = (180, 150, 240)
    r_lb2 = (0, 25, 130)
    r_ub2 = (20, 150, 240)
    
    red_exists, r_cont_area, r_cont_x = check_contours(image, r_lb1, r_ub1, r_lb2, r_ub2)

    # Define lower and upper bounds for blue in HSV
    b_lb1 = (100, 25, 130)
    b_ub1 = (130, 150, 240)
    b_lb2 = (110, 25, 130)  # Adjust these if a second range is needed
    b_ub2 = (140, 150, 240)
    
    # Check for blue contours
    blue_exists, b_cont_area, b_cont_x = check_contours(image, b_lb1, b_ub1, b_lb2, b_ub2)
    
    #if box_x == None:   
        #blue_exists = False
    
    # CASE 1: Obstacle(s) are visible and close --> move away L/R, pref center
    if red_exists and r_cont_area > 4000:
        print("Avoiding obstacle...")        
        print("r_cont_x:", r_cont_x)
        
        if r_cont_x > width/2 - margin and r_cont_x < width/2 + margin:
            # preference towards middle
            
            if r_cont_x > width/2:
                y = y + delta
                print("moving left")
            else:
                y = y - delta
                print("moving right")
    

   # CASE 2: Final goal in sight --> center image
    elif blue_exists and b_cont_area:
        print("Seeing Bird:")
        #print("w:", box_width, "h:", box_height)
        if box_width>= 0.25 or box_height>= 0.175:
            print("Landing...")
            return True, x, y
      
        err = 0.075
        delta = 0.02
        
        if box_x > err:
            y = y - delta
            print("moving right")
        elif box_x < -1 * err:
            y = y + delta
            print("moving left")

   # CASE 3: else go forward??.
    else: 
        x = x + delta
        print("moving forward")
     
    y_command = y
    x_command = x

    # set new position, hover at 0.85 meters
    cf.commander.send_position_setpoint(x_command, y_command, 0.85, 0) # Do not edit this line
    return False, x_command, y_command
 

## Test obstacle avoidance on the CrazyFlie ##

The following cell *will* fly the drone. Place the CrazyFlie in front of an obstacle in the netted area for testing. This cell will perform object detection and avoidance using the red filtering defined in the helper functions above.

In [48]:
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        cap = cv2.VideoCapture(camera)
        

        while(cap.isOpened()):

            ret, frame = cap.read()


            elapsed = time.time() - t
            if(elapsed > 5.0):

                print('Capturing.....')

                if ret:
                    #cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                    else:

                        if(check_contours(frame)):
                            current_y = adjust_position(cf, current_y)

            if(elapsed > 10.0):
                        break

        cap.release()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/18/2M
radio://0/101/2M
radio://0/105/2M
radio://0/18/2M
radio://0/20/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/105/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/105/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/

TypeError: adjust_position() missing 6 required positional arguments: 'box_x', 'box_y', 'box_width', 'box_height', 'current_x', and 'current_y'

In [19]:
import cv2
import numpy as np

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

In [51]:
import time

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

# ************ Parameters that might be useful to change ************ 
# COCO label id that we want to track
tracking_label = 2  #BIRD = 16, BICYCLE = 2

# Set the URI the Crazyflie will connect to
group_number = 18
uri = f'radio://0/{group_number}/2M'

# Possibly try 0, 1, 2 ...
camera = 1

# Confidence of detection (of the bird image)
confidence = 0.4

# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        ret, prev_frame = cap.read()
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        x_cur = 0
        y_cur = 0
        
        # detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            
            if ret:
                # image = (frame + prev_frame)/2
                # prev_frame = frame
                image = frame
                image_height, image_width, _ = image.shape
                
                # create blob from image
                blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                                        swapRB=True)

                # forward propagate image
                model.setInput(blob)
                detections = model.forward()

                # select detections that match selected class label
                matching_detections = [d for d in detections[0, 0] if d[1] == tracking_label]

                # select confident detections
                confident_detections = [d for d in matching_detections if d[2] > confidence]

                # get detection closest to center of field of view and draw it
                det = closest_detection(confident_detections) # This relies on the function you wrote above


                if det is not None:
                # get the class id
                    class_id = det[1]
                    # map the class id to the class 
                    class_name = class_names[int(class_id)-1]
                    color = COLORS[int(class_id)]
                    # get the bounding box coordinates
                    box_x = det[3] * image_width
                    box_y = det[4] * image_height
                    # get the bounding box width and height
                    box_width = det[5] * image_width
                    box_height = det[6] * image_height
                    # draw a rectangle around each detected object
                    cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
                    # put the class name text on the detected object
                    cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                    _, _, _, box_x, box_y, box_width, box_height = det
                    box_x, box_y = detection_center(det)
                    
                else:
                    box_x = None
                    box_y = None
                    box_width = None
                    box_height = None

                exit_loop, x_cur, y_cur = adjust_position(cf, image, box_x, box_y, box_width, box_height, x_cur, y_cur)

                # Check image
                cv2.imshow('image', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
            else:
                print('no image!!')
                
        cap.release()
        
        # Descend and stop all motion:
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/20/2M
radio://0/101/2M
radio://0/18/2M
radio://0/101/2M
radio://0/18/2M
radio://0/101

In [47]:
cap.release()